In [17]:
from z3 import *
import numpy as np
from utils2 import *

# Example function to create the Z3 constraints for MCP
def mcp_st(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Sizes of packages
    max_nodes = instance["max_nodes"]  # max_nodes horizon

    solver = Solver()
    solver.set("timeout", timeout)

    # Variables
    # To codify that courier i delivers package j at max_nodes k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(max_nodes + 1)] for j in range(n + 1)] for i in range(m)]

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range(m):
        weight_set = []
        for j in range(n):
            for k in range(1, max_nodes):
                for _ in range(s[j]):
                    weight_set.append(v[i][j][k])
        solver.add(at_most_k_seq(weight_set, min(l[i], max(s)), f"courier_{i}_load"))

    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(v[i][n][0])  # Start at position n at max_nodes 0
        solver.add(v[i][n][max_nodes])  # End at position n at max_nodes max_nodes

    # 3. Each courier cannot be in two places at the same max_nodes
    for i in range(m):
        for k in range(max_nodes + 1):
            solver.add(exactly_one_bw([v[i][j][k] for j in range(n + 1)], f"amo_package_{i}_{k}"))

    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_bw([v[i][j][k] for k in range(1, max_nodes) for i in range(m)], f"exactly_once_{j}"))

    # 5. If each courier must deliver at least one package
    for i in range(m):
        solver.add(Or([v[i][j][1] for j in range(n)]))

    return solver, v

# Example function to add distance constraint
def add_distance_constraint_standard(solver, instance, v, upperBound):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_nodes = instance["max_nodes"]  # max_nodes horizon
    distances = instance["D"]  # Distances between packages

    for i in range(m):
        dists = []
        for start in range(n + 1):
            for end in range(n + 1):
                for z in range(distances[start][end]):
                    dists.append(Or(Not(v[i][start][end]), BoolVal(True)))
        solver.add(at_most_k_seq(dists, upperBound, f"Courier_dist_{i}_{upperBound}"))

    # Ensure that the number of nodes is not exceeded
    if np.sum(distances[:max_nodes]) <= upperBound:
        for i in range(m):
            solver.add(v[i][n][max_nodes + 1])

# Example function to solve the MCP using Z3
def run_sat_linear_search(instance):
    # Copy of the instance since we will modify it
    instance = dict(instance)

    # Initialize the Z3 solver and variables
    solver, v = mcp_st(instance)

    # Set the original upper bound and lower bound for linear search
    original_upper_bound = instance["max_dist"]
    lower_bound = instance["min_dist"]
    upper_bound = original_upper_bound

    # Step size for the linear search (can be adjusted)
    step = 10

    # Linear search for finding a feasible solution
    distance_limit = lower_bound
    while distance_limit <= upper_bound:
        solver.push()

        # Add the distance constraint to the solver
        add_distance_constraint_standard(solver, instance, v, distance_limit)

        # Check if the problem is satisfiable
        if solver.check() == sat:
            model = solver.model()
            solution = extract_solution(model, v, instance["m"], instance["n"], instance["max_nodes"])
            return solution, objective_function(solution, instance)

        # Increment the distance limit for the next iteration
        distance_limit += step

        # Pop the constraint for the next iteration
        solver.pop()

    # Return unsat if no solution found within the distance constraints
    return "unsat", None

# Example function to extract the solution from Z3 model
def extract_solution(model, v, m, n, max_nodes):
    solution = []

    for i in range(m):
        courier_solution = []
        for k in range(1, max_nodes + 1):
            for j in range(n):
                if model[v[i][j][k]]:
                    courier_solution.append(j + 1)
        solution.append(courier_solution)

    return solution

# Example objective function: total distance traveled by all couriers
def objective_function(solution, instance):
    total_distance = 0
    distances = instance["D"]
    n = instance["n"]

    for courier_solution in solution:
        if not courier_solution:
            continue
        depot = n  # Starting node
        for node in courier_solution:
            total_distance += distances[depot][node]
            depot = node
        total_distance += distances[depot][n]  # Return to starting node

    return total_distance

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_nodes steps
    max_nodes = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_nodes": max_nodes,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Example instance data
instance = {
    "m": 3,  # Number of couriers
    "n": 4,  # Number of packages
    "l": [15, 20, 25],  # Weights each courier can carry
    "s": [5, 8, 7, 6],  # Sizes of packages
    "D": [
        [0, 10, 20, 15, 10],
        [10, 0, 25, 30, 20],
        [20, 25, 0, 35, 15],
        [15, 30, 35, 0, 25],
        [10, 20, 15, 25, 0],
    ],  # Distances between packages
}

# Compute additional info based on the instance
additional_info = compute_additional_info(instance)
instance.update(additional_info)

# Run the linear search SAT solver
result, obj_value = run_sat_linear_search(instance)

# Process the final result
if result == "unsat":
    print("No solution found within the distance constraints.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx + 1}: {path}")

    print(f"Objective Function Value: {obj_value}")


IndexError: list index out of range

## binary search

In [18]:
from z3 import *
from utils2 import *
import numpy as np

def mcp_st(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Sizes of packages
    max_nodes = instance["max_nodes"]  # max_nodes horizon
    min_load = instance["min_load"]  # Minimum load each courier can carry
    max_load = instance["max_load"]  # Maximum load each courier can carry
    equal_load_matrix = instance["equal_matrix"]

    solver = Solver()
    solver.set("timeout", timeout)

    # Variables
    # To codify that courier i delivers package j at max_nodes k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(max_nodes + 1)] for j in range(n + 1)] for i in range(m)]

    # To codify that courier i goes from node start to node end
    d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n + 1)]
          for start in range(n + 1)] for i in range(m)]

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range(m):
        weight_set = []
        for j in range(n):
            for k in range(1, max_nodes):
                for _ in range(s[j]):
                    weight_set.append(v[i][j][k])
        solver.add(at_most_k_seq(weight_set, min(l[i], max_load), f"courier_{i}_load"))

    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(v[i][n][0])  # Start at position n at max_nodes 0
        solver.add(v[i][n][max_nodes])  # End at position n at max_nodes max_nodes

    # 3. Each courier cannot be in two places at the same max_nodes
    for i in range(m):
        for k in range(max_nodes + 1):
            solver.add(exactly_one_bw([v[i][j][k] for j in range(n + 1)], f"amo_package_{i}_{k}"))

    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_bw([v[i][j][k] for k in range(1, max_nodes) for i in range(m)], f"exactly_once_{j}"))

    # 5. If each courier must deliver at least one package
    for i in range(m):
        solver.add(Or([v[i][j][1] for j in range(n)]))

    return solver, v, d

def add_distance_constraint_standard(solver, instance, v, d, upperBound):
    m = instance["m"]  # couriers
    n = instance["n"]  # packages
    max_nodes = instance["max_nodes"]  # max_nodes
    distances = instance["D"]  # distances between packages
    for i in range(m):
        dists = []
        for start in range(n + 1):
            for end in range(n + 1):
                for z in range(distances[start][end]):
                    dists.append(d[i][start][end])
        solver.add(at_most_k_seq(dists, upperBound, f"Courier_dist_{i}_{upperBound}"))
    
    flattened_distances = np.array(instance['D']).flatten()
    flattened_distances = flattened_distances[flattened_distances != 0]
    flattened_distances = np.sort(flattened_distances)

    max_k = 0

    while np.sum(flattened_distances[:max_k]) <= upperBound and max_k < n:
        max_k += 1
        
    if max_k < max_nodes:
        for i in range(m):
            solver.add(v[i][n][max_k + 1])

def run_sat(instance):
    # copy of the instance since we will modify it
    instance = dict(instance)
    mcp = mcp_st
    add_constraint = add_distance_constraint_standard

    # add additional info to the instance
    add_additional_info(instance)
    # original upper bound -> used in the binary search to check if the problem is unsat
    original_upper_bound = instance["max_dist"]
    # lower bound for the binary search
    lower_bound = instance["min_dist"]
    # upper bound for the binary search
    upper_bound = original_upper_bound

    # pivot for the binary search
    pivot = (original_upper_bound + lower_bound) // 2

    # solver creation based on the model used
    solver, v, d = mcp(instance) 

    res = None

    best_solution = None

    # binary search using bounds
    while True:
        # if the lower bound is equal to the upper bound, we have found the optimal solution
        if lower_bound >= upper_bound and res is not None:
            return best_solution, objective_function(best_solution, instance)

        # push/pop mechanism for adding new constraint and solving the problem
        solver.push()

        # add the constraint to the solver to force the distance to be lower or equal than the pivot
        add_constraint(solver, instance, v, d, pivot)

        # solve the problem with the current constraints
        res = solve_mcp(solver)

        # if the problem is unsat
        if res == unsat:
            # if we never found a solution then upper bound is the same as the original upper bound hence the problem is unsat
            if upper_bound == original_upper_bound:
                # if the upper bound is equal to the original upper bound, the problem is unsat since we never found a solution
                return "unsat", None
            
            # updating lower
            lower_bound = pivot + 1
            # pop the constraint since we will add a new one
            solver.pop()
            # update the pivot
            pivot = (upper_bound + lower_bound) // 2
        else:
            # if the problem is sat, we save the solution
            best_solution = format_solution(instance['m'], instance['n'], instance['max_nodes'], res, v)
            upper_bound = pivot
            pivot = (upper_bound + lower_bound) // 2


def add_additional_info(instance):
    m = instance["m"]
    l = instance["l"]
    equal_matrix = np.full((m, m), BoolVal(False))
    for i in range(m):
        for i1 in range(m):
            if l[i] == l[i1]:
                equal_matrix[i, i1] = BoolVal(True)
    instance["equal_matrix"] = equal_matrix

def solve_mcp(solver):
    if solver.check() == sat:
        return solver.model()
    else:  
        return unsat

def format_solution(m, n, max_nodes, model, v):
    solution = []

    for i in range(m):
        courier_solution = []

        for k in range(max_nodes):
            for j in range(n):
                if model[v[i][j][k]]:
                    courier_solution.append(j + 1)

        solution.append(courier_solution)

    return solution

def objective_function(solution, instance):
    # Example objective function: total distance traveled by all couriers
    total_distance = 0
    distances = instance["D"]
    n = instance["n"]

    for courier_solution in solution:
        if not courier_solution:
            continue
        depot = n  # Starting node
        for node in courier_solution:
            total_distance += distances[depot][node]
            depot = node
        total_distance += distances[depot][n]  # Return to starting node

    return total_distance

def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]
    
    # Number of max_nodes steps
    max_nodes = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load 
    #max_load = np.sum(np.sort(instance["s"])[-max_nodes:])
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    min_distance = 0

    #print(max_nodes, min_load, max_load, min_distance, max_distance)

    additional_info = {
        "max_nodes": max_nodes,
        "max_dist": max_distance,
        "min_dist": min_distance,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

#################################################################################

# choose inst.
NUM_INST = 5
instance = instances[NUM_INST - 1]

# Compute additional info based on the instance
additional_info = compute_additional_info(instance)
instance.update(additional_info)

# Run the SAT solver
result, obj_value = run_sat(instance)

# Process the final result
if result == "unsat":
    print("No solution found.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx}: {path}")

    print(f"Objective Function Value: {obj_value}")

No solution found.


In [16]:
import os

def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

def read_all_dat_files(directory):
    instances = []
    for filename in os.listdir(directory):
        if filename.endswith('.dat'):
            file_path = os.path.join(directory, filename)
            instance = read_dat_file(file_path)
            instances.append(instance)
    return instances

# Directory containing .dat files
directory = 'instances'

# Read all .dat files and populate instances
instances = read_all_dat_files(directory)